# INSEE income data - User validation 

In [1]:
import twitter_profile_predictor as tpp

import pandas as pd
import numpy as np

[nltk_data] Downloading package punkt to
[nltk_data]     /home/livtollanes/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/livtollanes/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
import matplotlib 

ModuleNotFoundError: No module named 'matplotlib'

In [53]:
# # Load the data

# path = '/home/livtollanes/NewData/coordinates/dim_X/'
# file = '/m1_dimx_users_bios.csv'
# m1_bios = pd.read_csv(path+file, index_col=0)


# If the job titles have already been collected
path = '/home/livtollanes/NewData/professions/'
file = 'm1_professions.csv'
m1_bios = pd.read_csv(path+file, index_col=0)


In [3]:
#analyser = tpp.df_bios_analyzer(m1_bios, description_column= 'description_cleantext')
#analyser.get_all(mainfrench='True')
#analyser.get_PCSgroup()

# #write to csv to avoid rerunning every time
# m1_bios.to_csv('/home/livtollanes/NewData/professions/m1_professions.csv')

,follower_id,0,1,2,projection,screen_name,description_cleantext,tokens,bi_tokens,full_tokens,professions,prostatus,actorstatus,groupstatus,universitystatus,allstatus,age,gender,topic,lang
0,1384090711327350784,-1.857364,3.529058,0.575453,2.087315,ObjectifDataSci,"J'aide ceux qui se forment à la #datascience, ...","[jaide, ceux, forment, datascience, machinelea...","[(jaide, ceux), (ceux, forment), (forment, dat...","[jaide, ceux, forment, datascience, machinelea...",[],[],[],[],[],[],[],[],[],fr
1,4864108900,-1.847305,3.532449,0.605202,2.083836,Smartmechanics,La mécanique pour l'Industrie du futur. Le rés...,"[mécanique, lindustrie, futur, réseau, smartme...","[(mécanique, lindustrie), (lindustrie, futur),...","[mécanique, lindustrie, futur, réseau, smartme...",[],[],[],[],[],[],[],[],[],fr
2,3726109276,-1.847305,3.532449,0.605202,2.083836,BigDataAristote,Ce compte est inactif depuis le 22 décembre 20...,"[compte, inactif, depuis, 22, décembre, 2016, ...","[(compte, inactif), (inactif, depuis), (depuis...","[compte, inactif, depuis, 22, décembre, 2016, ...",[],[],[],[],[],[],[],[],[],fr
3,2825786052,-1.847305,3.532449,0.605202,2.083836,benextcompany,"BENEXT devient OCTO Technology, une société de...","[benext, devient, octo, technology, société, c...","[(benext, devient), (devient, octo), (octo, te...","[benext, devient, octo, technology, société, c...",[],[specialist],[],[],[],[specialist],[],[],"[technology, development, digital]",fr
4,147610756,-1.847305,3.532449,0.605202,2.083836,StephetFred,HPE Education Services France vous propose des...,"[hpe, education, services, france, propose, fo...","[(hpe, education), (education, services), (ser...","[hpe, education, services, france, propose, fo...",[],[],[],[],[],[],[],[],"[technology, education]",fr
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115074,3503654182,2.276890,1.233403,-2.291264,-2.649039,SportsValeurs,Partageons les bonnes pratiques du sports ! Su...,"[partageons, bonnes, pratiques, sports, suivez...","[(partageons, bonnes), (bonnes, pratiques), (p...","[partageons, bonnes, pratiques, sports, suivez...",[],[],[],[],[],[],[],[],[],fr
115075,1430233254016307200,2.301663,1.264603,-2.292688,-2.672192,JulieDfco21,Fan du rc Lens et du Dfco CM: @100_DFCO_NEWS,"[fan, rc, lens, dfco, cm, 100dfconews]","[(fan, rc), (rc, lens), (lens, dfco), (dfco, c...","[fan, rc, lens, dfco, cm, 100dfconews, (fan, r...",[],[],[],[fan],[],[fan],[],[],[],fr
115076,1251551779,2.309714,1.263744,-2.292223,-2.679998,Oxygene_tango,WebRadio officielle du @stadelavallois // htt...,"[webradio, officielle, stadelavallois, httpwww...","[(webradio, officielle), (officielle, stadelav...","[webradio, officielle, stadelavallois, httpwww...",[],[],[],[],[],[],[],[],[],fr
115077,785463278977839104,2.341063,1.305582,-2.380031,-2.727126,masterdiete,le cheat meal le meilleur des reconfort apres ...,"[cheat, meal, meilleur, reconfort, apres, sema...","[(cheat, meal), (meal, meilleur), (meilleur, r...","[cheat, meal, meilleur, reconfort, apres, sema...",[],[],[],[],[],[],[],[],[],fr


In [54]:
#Select only rows where professions is not an empty list
m1_bios_professions = m1_bios[m1_bios['professions'].apply(lambda x: x != "[]")] #removing 87 183 rows

In [55]:
m1_bios_professions.shape #27896
#m1_bios.shape #115 079

(27896, 21)

In [8]:
pd.set_option('display.max_colwidth', None)
columns = ['projection', 'description_cleantext', 'professions','allstatus', 'topic', 'PCSgroup']
m1_bios_professions[columns].head(30)

,projection,description_cleantext,professions,allstatus,topic,PCSgroup
8,2.083836,"Consultant, formateur et animateur en oenologiechez Dégustation et Harmonie""Mieux analyser le vin pour laisser parler vos émotions""","['animator', 'professor', 'consultant']",[],[],"['Professors and higher scientific professions', 'Information, arts and entertainment professions', 'Business, IT and administration professionals']"
9,2.083836,Directeur Général (CEO) de @EasywaFR contact pro : melvyn@easywa.fr,['ceo'],"['professional', 'director']",[],"['Business, IT and administration professionals']"
12,2.073843,HR Development #RH #Recrutement #Formation #Communication #MarqueEmployeur #Digital Fan de rugby,['human resources'],['fan'],"['recruitment', 'communication', 'digital', 'development', 'education']","['Business, IT and administration professionals']"
17,1.993296,Je suis convaincu que l'#education est la base du progrès et la #RH la clef du succès de l'#entreprise. #lyon #france #europe,['human resources'],[],['recruitment'],"['Business, IT and administration professionals']"
26,1.982493,Union des Professeurs de Sciences et Techniques Industrielles. 700 profs de Sciences de l’Ingénieur & Informatique notamment en classe prépa scientifiques.,['engineer'],[],"['digital', 'sciences']","['Business, IT and administration professionals']"
46,1.950943,"Doctorante sciences de l'éducation - CREADIngénierie pédagogique - Arts & Métiers Interaction enseignant-étudiant, pédagogie universitaire",['researcher'],['phd'],"['sciences', 'culture']",['Professors and higher scientific professions']
50,1.948695,"ESN spécialisée en dév #informatique, nous accompagnons nos clients grâce à nos Talents JAVA, .NET, mobile, Business Analyst, BI, PHP, Big Data...",['analyst'],['specialist'],"['data', 'digital', 'business']","['Business, IT and administration professionals']"
57,1.943129,Responsable de la communication scientifique de l'École polytechnique,['communications manager'],['executive'],"['communication', 'sciences']","['Business, IT and administration professionals']"
64,1.932874,Chercheur à @institut_curie et professeur @cdf1530,"['researcher', 'professor']",[],['research'],['Professors and higher scientific professions']
78,1.910042,Directrice Communication et Marque du Pôle Léonard de Vinci #poledevinci #HigherEd #EdTech #Paris @poledevinci - VP @ellesbougent#emojibzh,['communications manager'],['director'],['communication'],"['Business, IT and administration professionals']"


In [56]:
pd.set_option('display.max_rows', None)
m1_bios['PCSgroup'].value_counts()

PCSgroup
[]                                                                                                                                                                                                        87183
['Information, arts and entertainment professions']                                                                                                                                                       11993
['Business, IT and administration professionals']                                                                                                                                                          6060
['Elected officers and political representatives']                                                                                                                                                         2773
['Professors and higher scientific professions']                                                                                                               

In [57]:
import ast

#Filter the DataFrame to only include the users with one title only
m1_single_jobs = m1_bios_professions.loc[m1_bios_professions['professions'].apply(lambda x: len(ast.literal_eval(x)) == 1)]

In [58]:
m1_single_jobs['professions'] = m1_single_jobs['professions'].apply(lambda x: ast.literal_eval(x)[0])
m1_single_jobs['PCSgroup'] = m1_single_jobs['PCSgroup'].apply(lambda x: ast.literal_eval(x)[0])

/tmp/ipykernel_2841783/1386365019.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m1_single_jobs['professions'] = m1_single_jobs['professions'].apply(lambda x: ast.literal_eval(x)[0])
/tmp/ipykernel_2841783/1386365019.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m1_single_jobs['PCSgroup'] = m1_single_jobs['PCSgroup'].apply(lambda x: ast.literal_eval(x)[0])


In [61]:
m1_single_jobs['professions'].value_counts()

professions
journalist                5573
author                    1808
professor                 1782
consultant                 838
editor                     829
communications manager     797
ceo                        715
attorney                   630
engineer                   629
entrepreneur               604
city ​​councilor           574
human resources            509
researcher                 471
photographer               453
artist                     394
mayor                      367
lawyer                     343
project manager            328
coach                      314
animator                   307
nurse                      297
blogger                    289
member of parliament       272
actor                      255
developer                  250
designer                   215
physician                  190
regional councilor         189
producer                   172
mission manager            151
film director              149
departmental councilor     

In [104]:

col = 'departmental councilor'  # replace with the value you're interested in
m1_single_jobs.loc[m1_single_jobs['professions'].apply(lambda x: x == col)].iloc[10:15]

,follower_id,0,1,2,projection,screen_name,description_cleantext,tokens,bi_tokens,full_tokens,professions,prostatus,actorstatus,groupstatus,universitystatus,allstatus,age,gender,topic,lang,PCSgroup
15461,589126664,-0.689717,-0.441905,-0.440002,0.563067,Departement54,Fil d'actualités du conseil départemental de Meurthe-et-Moselle (compte officiel),"['fil', 'dactualités', 'conseil', 'départemental', 'meurtheetmoselle', 'compte', 'officiel']","[('fil', 'dactualités'), ('dactualités', 'conseil'), ('conseil', 'départemental'), ('départemental', 'meurtheetmoselle'), ('meurtheetmoselle', 'compte'), ('compte', 'officiel')]","['fil', 'dactualités', 'conseil', 'départemental', 'meurtheetmoselle', 'compte', 'officiel', ('fil', 'dactualités'), ('dactualités', 'conseil'), ('conseil', 'départemental'), ('départemental', 'meurtheetmoselle'), ('meurtheetmoselle', 'compte'), ('compte', 'officiel')]",departmental councilor,[],['official account'],[],[],['official account'],[],[],[],fr,Elected officers and political representatives
15806,3248872427,-0.628562,0.012961,-0.282868,0.555738,udijeunes86,Compte Twitter des jeunes UDI de la Vienne #TeamUDIJeunes86 #TeamKiwi Délégué Départemental @AlexandreCadu #UDIJeunes,"['compte', 'twitter', 'jeunes', 'udi', 'vienne', 'teamudijeunes86', 'teamkiwi', 'délégué', 'départemental', 'alexandrecadu', 'udijeunes']","[('compte', 'twitter'), ('twitter', 'jeunes'), ('jeunes', 'udi'), ('udi', 'vienne'), ('vienne', 'teamudijeunes86'), ('teamudijeunes86', 'teamkiwi'), ('teamkiwi', 'délégué'), ('délégué', 'départemental'), ('départemental', 'alexandrecadu'), ('alexandrecadu', 'udijeunes')]","['compte', 'twitter', 'jeunes', 'udi', 'vienne', 'teamudijeunes86', 'teamkiwi', 'délégué', 'départemental', 'alexandrecadu', 'udijeunes', ('compte', 'twitter'), ('twitter', 'jeunes'), ('jeunes', 'udi'), ('udi', 'vienne'), ('vienne', 'teamudijeunes86'), ('teamudijeunes86', 'teamkiwi'), ('teamkiwi', 'délégué'), ('délégué', 'départemental'), ('départemental', 'alexandrecadu'), ('alexandrecadu', 'udijeunes')]",departmental councilor,['delegate'],[],[],[],['delegate'],[],['Man'],[],fr,Elected officers and political representatives
16658,245710744,-0.605945,0.081657,-0.263976,0.540548,VAG_86,Groupe des élus de gauche du Conseil Départemental de la Vienne,"['groupe', 'élus', 'gauche', 'conseil', 'départemental', 'vienne']","[('groupe', 'élus'), ('élus', 'gauche'), ('gauche', 'conseil'), ('conseil', 'départemental'), ('départemental', 'vienne')]","['groupe', 'élus', 'gauche', 'conseil', 'départemental', 'vienne', ('groupe', 'élus'), ('élus', 'gauche'), ('gauche', 'conseil'), ('conseil', 'départemental'), ('départemental', 'vienne')]",departmental councilor,[],[],[],[],[],[],[],[],fr,Elected officers and political representatives
16779,1657941,-0.696638,-0.568611,-0.565005,0.538291,julanimtic,"Chargé de projets numériques - Conseil Départemental du Val de Marne bidouille, JV, Art numérique chronique JV pour la revue des livres pour enfants","['chargé', 'projets', 'numériques', 'conseil', 'départemental', 'val', 'marne', 'bidouille', 'jv', 'art', 'numérique', 'chronique', 'jv', 'revue', 'livres', 'enfants']","[('chargé', 'projets'), ('projets', 'numériques'), ('numériques', 'conseil'), ('conseil', 'départemental'), ('départemental', 'val'), ('val', 'marne'), ('marne', 'bidouille'), ('bidouille', 'jv'), ('jv', 'art'), ('art', 'numérique'), ('numérique', 'chronique'), ('chronique', 'jv'), ('jv', 'revue'), ('revue', 'livres'), ('livres', 'enfants')]","['chargé', 'projets', 'numériques', 'conseil', 'départemental', 'val', 'marne', 'bidouille', 'jv', 'art', 'numérique', 'chronique', 'jv', 'revue', 'livres', 'enfants', ('chargé', 'projets'), ('projets', 'numériques'), ('numériques', 'conseil'), ('conseil', 'départemental'), ('départemental', 'val'), ('val', 'marne'), ('marne', 'bidouille'), ('bidouille', 'jv'), ('jv', 'art'), ('art', 'numérique'), ('numérique', 'chronique'), ('chronique', 'jv'), ('jv', 'revue'), ('revue', 'livres'), ('livres', 'e

In [30]:
m1_single_jobs['PCSgroup'].value_counts()

PCSgroup
['Information, arts and entertainment professions']    10229
['Business, IT and administration professionals']       5642
['Professors and higher scientific professions']        2584
['Elected officers and political representatives']      1952
['Legal professions']                                    973
['Other']                                                863
Name: count, dtype: int64

In [52]:
m1_single_jobs['professions'].unique #we have 51 unique, single job titles 

<bound method Series.unique of 9                                ['ceo']
12                   ['human resources']
17                   ['human resources']
26                          ['engineer']
46                        ['researcher']
50                           ['analyst']
57            ['communications manager']
78            ['communications manager']
82                        ['consultant']
83                         ['professor']
87                 ['community manager']
97            ['communications manager']
102                  ['human resources']
104                  ['project manager']
107                  ['mission manager']
108                       ['accountant']
114                            ['actor']
115                  ['human resources']
119                  ['human resources']
122                            ['actor']
125                  ['human resources']
133                       ['consultant']
137                  ['mission manager']
139                       

In [32]:
pd.set_option('display.max_rows', None)
m1_single_jobs.groupby('PCSgroup')['professions'].value_counts()

PCSgroup                                             professions                   
['Business, IT and administration professionals']    ['consultant']                     838
                                                     ['communications manager']         797
                                                     ['ceo']                            715
                                                     ['engineer']                       629
                                                     ['entrepreneur']                   604
                                                     ['human resources']                509
                                                     ['project manager']                328
                                                     ['coach']                          314
                                                     ['developer']                      250
                                                     ['designer']                       

In [37]:
m1_single_jobs['professions'].value_counts()

professions
journalist                5573
author                    1808
professor                 1782
consultant                 838
editor                     829
communications manager     797
ceo                        715
attorney                   630
engineer                   629
entrepreneur               604
city ​​councilor           574
human resources            509
researcher                 471
photographer               453
artist                     394
mayor                      367
lawyer                     343
project manager            328
coach                      314
animator                   307
nurse                      297
blogger                    289
member of parliament       272
actor                      255
developer                  250
designer                   215
physician                  190
regional councilor         189
producer                   172
mission manager            151
film director              149
departmental councilor     

#Income test

In [121]:
#load income data
path = '/home/livtollanes/10.jan-thesis/labeldata/'
file = 'grov_betaling.csv'
income = pd.read_csv(path+file, sep=',')

#rename data_job_title to professions
income.rename(columns={'data_job_titles':'professions'}, inplace=True)

In [123]:
income.head()

,professions,french_title,PCS_group_tim,PCS_french,PCS_code,INSEE_label,Income_moyen_EQTP,income_mois,source
0,journalist,journaliste,"Information, arts and entertainment professions","Professions de l’information, de l’art et des spectacles",35B,NaN,NaN,"2 324,00",https://fr.talent.com/salary?job=journaliste
1,author,auteur/auteure,"Information, arts and entertainment professions","Professions de l’information, de l’art et des spectacles",35B,NaN,NaN,"3 125,00",https://fr.talent.com/salary?job=auteur
2,professor,professeur/professeure,Professors and higher scientific professions,Professeurs et professions scientifiques supérieures,34,NaN,NaN,"3 180,00",https://fr.talent.com/salary?job=professeur
3,consultant,consultant/consultante,"Business, IT and administration professionals","Cadres et professions intermédiaires administratives, commerciales et techniques des entreprises",NaN,NaN,NaN,"3 167,00",https://fr.talent.com/salary?job=consultant
4,editor,éditeur/éditrice,"Information, arts and entertainment professions","Professions de l’information, de l’art et des spectacles",35,NaN,NaN,"3 953,00",https://fr.talent.com/salary?job=%C3%A9diteur


In [124]:
m1_single_jobs = m1_single_jobs.merge(income[['professions', 'income_mois']], on='professions', how='left')

In [125]:
m1_single_jobs.head()

,follower_id,0,1,2,projection,screen_name,description_cleantext,tokens,bi_tokens,full_tokens,professions,prostatus,actorstatus,groupstatus,universitystatus,allstatus,age,gender,topic,lang,PCSgroup,income_mois
0,728612213339115520,-1.847305,3.532449,0.605202,2.083836,MelvynBoukli,Directeur Général (CEO) de @EasywaFR contact pro : melvyn@easywa.fr,"['directeur', 'général', 'ceo', 'easywafr', 'contact', 'pro', 'melvyneasywafr']","[('directeur', 'général'), ('général', 'ceo'), ('ceo', 'easywafr'), ('easywafr', 'contact'), ('contact', 'pro'), ('pro', 'melvyneasywafr')]","['directeur', 'général', 'ceo', 'easywafr', 'contact', 'pro', 'melvyneasywafr', ('directeur', 'général'), ('général', 'ceo'), ('ceo', 'easywafr'), ('easywafr', 'contact'), ('contact', 'pro'), ('pro', 'melvyneasywafr')]",ceo,"['professional', 'director']",[],[],[],"['professional', 'director']",[],['Man'],[],fr,"Business, IT and administration professionals","4 167,00"
1,533259254,-1.851397,3.473119,0.550395,2.073843,delphine_tweet,HR Development #RH #Recrutement #Formation #Communication #MarqueEmployeur #Digital Fan de rugby,"['hr', 'development', 'rh', 'recrutement', 'formation', 'communication', 'marqueemployeur', 'digital', 'fan', 'rugby']","[('hr', 'development'), ('development', 'rh'), ('rh', 'recrutement'), ('recrutement', 'formation'), ('formation', 'communication'), ('communication', 'marqueemployeur'), ('marqueemployeur', 'digital'), ('digital', 'fan'), ('fan', 'rugby')]","['hr', 'development', 'rh', 'recrutement', 'formation', 'communication', 'marqueemployeur', 'digital', 'fan', 'rugby', ('hr', 'development'), ('development', 'rh'), ('rh', 'recrutement'), ('recrutement', 'formation'), ('formation', 'communication'), ('communication', 'marqueemployeur'), ('marqueemployeur', 'digital'), ('digital', 'fan'), ('fan', 'rugby')]",human resources,[],[],['fan'],[],['fan'],[],[],"['recruitment', 'communication', 'digital', 'development', 'education']",fr,"Business, IT and administration professionals","2 401,00"
2,3752085807,-1.784780,3.367104,0.498231,1.993296,henri_meisner,Je suis convaincu que l'#education est la base du progrès et la #RH la clef du succès de l'#entreprise. #lyon #france #europe,"['convaincu', 'leducation', 'base', 'progrès', 'rh', 'clef', 'succès', 'lentreprise', 'lyon', 'france', 'europe']","[('convaincu', 'leducation'), ('leducation', 'base'), ('base', 'progrès'), ('progrès', 'rh'), ('rh', 'clef'), ('clef', 'succès'), ('succès', 'lentreprise'), ('lentreprise', 'lyon'), ('lyon', 'france'), ('france', 'europe')]","['convaincu', 'leducation', 'base', 'progrès', 'rh', 'clef', 'succès', 'lentreprise', 'lyon', 'france', 'europe', ('convaincu', 'leducation'), ('leducation', 'base'), ('base', 'progrès'), ('progrès', 'rh'), ('rh', 'clef'), ('clef', 'succès'), ('succès', 'lentreprise'), ('lentreprise', 'lyon'), ('lyon', 'france'), ('france', 'europe')]",human resources,[],[],[],[],[],[],[],['recruitment'],fr,"Business, IT and administration professionals","2 401,00"
3,2885598352,-1.799371,3.132626,0.426654,1.982493,upstifr,Union des Professeurs de Sciences et Techniques Industrielles. 700 profs de Sciences de l’Ingénieur & Informatique notamment en classe prépa scientifiques.,"['union', 'professeurs', 'sciences', 'techniques', 'industrielles', '700', 'profs', 'sciences', '’', 'ingénieur', 'informatique', 'notamment', 'classe', 'prépa', 'scientifiques']","[('union', 'professeurs'), ('professeurs', 'sciences'), ('sciences', 'techniques'), ('techniques', 'industrielles'), ('industrielles', '700'), ('700', 'profs'), ('profs', 'sciences'), ('sciences', '’'), ('’', 'ingénieur'), ('ingénieur', 'informatique'), ('informatique', 'notamment'), ('notamment', 'classe'), ('classe', 'prépa'), ('prépa', 'scientifiques')]","['union', 'professeurs', 'sciences', 'techniques', 'industrielles', '700', 'profs', 'sciences', '’', 'ingénieur', 'informatique', 'notamment', 'classe', 'prépa', 'scientifiques', ('union', 'professeurs'), ('professeurs', 'sciences'), ('sciences',

In [127]:
import matplotlib.pyplot as plt
import numpy as np

# Calculate the median, its standard deviation, and mean of 'income_mois' for each profession
grouped_subset = m1_single_jobs.groupby('professions').agg({'income_mois': ['median', 'std', 'mean']}).reset_index()

# Sort the professions by median income
grouped_subset = grouped_subset.sort_values(('income_mois', 'median'))

plt.figure(figsize=(10, 12))  # Increase the height of the figure

# Plot error bars for each profession with a small random jitter added to the y-coordinates
for i in range(grouped_subset.shape[0]):
    jitter = np.random.uniform(-0.3, 0.3)  # adjust the range as needed
    plt.errorbar(x=grouped_subset[('income_mois', 'median')][i], 
                 y=grouped_subset[('income_mois', 'mean')][i] + jitter, 
                 xerr=grouped_subset[('income_mois', 'std')][i], 
                 fmt='o')

# Add profession labels to each point
for i in range(grouped_subset.shape[0]):
    plt.text(grouped_subset[('income_mois', 'median')][i], 
             grouped_subset[('income_mois', 'mean')][i], 
             grouped_subset['professions'][i], 
             ha = 'right', va = 'bottom')

plt.xlabel('Median Estimated Income')
plt.ylabel('Mean Monthly Income')
plt.title('Median Estimated Income vs Mean Monthly Income by Profession')
plt.show()

ModuleNotFoundError: No module named 'matplotlib'

In [128]:
import matplotlib 

ModuleNotFoundError: No module named 'matplotlib'